# Correlation Testing

In [6]:
import os
from pricepredict import PricePredict

# Get all daily PPO objects from the ppo dir
ppos = {}
for file in os.listdir('../ppo'):
    # Check if filename has _D_ in it and ends with .dill
    if '_D_' in file and file.endswith('.dill'):
        # Get the symbol name from the file name (first chars before _D_)
        symbol = file.split('_D_')[0]
        # Load the PPO object from the file
        with open(f'../ppo/{file}', 'rb') as f:
            pp_obj = f.read()
        # unserialize the PPO object
        ppos[symbol] = PricePredict.unserialize(pp_obj)

In [7]:

all_symbols = sorted(ppos.keys())

# Loop through the ppos sorted by key (symbol)
for symbol in all_symbols:
    print(f'{ppos[symbol].ticker}')
    # Generate correlations between each symbol and all other symbols
    for symbol2 in all_symbols:
        if symbol != symbol2:
            # Get the corr between the two symbols
            correlation = ppos[symbol].periodic_correlation(ppos[symbol2], pc_period_len=30)
            print(f'{symbol2}: {correlation['coint_stationary']}')


pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] i

000001.SS
AAOI: False
AAPL: False
ABT: False
ACAD: False
ACN: False
ADBE: False
ADM: False
ADP: False
AIG: False
ALGM: False
ALKS: False
ALL: False
AMGN: False
AMX: False
AMZN: False
ANTM.JK: False
AON: False
APD: False
APTV: False
ASND: False
AVGO: False
AXON: False
AXP: False
BA: False
BB: False
BCRX: False
BFH: False


pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] is less than the minimum length of the data [240]. self_len[241], ppd_len[250]
pc_period_len [30] i

BIDU: False
BKNG: False
BLK: False
BMY: False
BRK-B: False
CAT: False
CB: False
CCI: False
CI: False
CL: False
CMCSA: False
CNC: False
CNMD: False
CRM: False
CRSP: False
CSCO: False
CSX: False
CVX: False
DHR: False
DIS: False
DUK: False
ECL: False
EGAN: False
EL: False
EMR: False
EOG: False
ERIC: False


Error: In periodic_correlation(): ADF Test - self.ticker[000001.SS] ppo.ticker[EURUSD=X] period[D]
exog contains inf or nans


ValueError: Error: Error: In periodic_correlation(): ADF Test - self.ticker[000001.SS] ppo.ticker[EURUSD=X] period[D]
exog contains inf or nans